In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator   #For data augmentation
import numpy as np

from keras.models import load_model
from keras.preprocessing import image

from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import SGD

import cv2

In [0]:
Optimizer = tf.keras.optimizers.RMSprop()
nb_classes = 10

label_names = {
    '0': 'Airplane',
    '1': 'Automobile',
    '2': 'Bird',
    '3': 'Cat',
    '4': 'Deer',
    '5': 'Dog',
    '6': 'Frog',
    '7': 'Horse',
    '8': 'Ship',
    '9': 'truck'
}

In [0]:
#To get pre-processes train and test data

def load_data():
  (x_train,y_train),(x_test,y_test) = datasets.cifar10.load_data()
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

  #Normalize
  mean = np.mean(x_train,axis=(0,1,2,3))
  std = np.std(x_train,axis=(0,1,2,3))
  x_train = (x_train - mean)/ (std+1e-7)
  x_test = (x_test - mean)/ (std+1e-7)

  #Format labels (OHE)
  y_train = tf.keras.utils.to_categorical(y_train, num_classes=nb_classes)
  y_test = tf.keras.utils.to_categorical(y_test, num_classes=nb_classes)
  return x_train , y_train , x_test, y_test

In [0]:
#Data augmentation

datagen = ImageDataGenerator(rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)
datagen.fit(x_train)

In [0]:
def build_model():
  model = models.Sequential()

  # CONV ==  BN == CONV == BN == MAXPOOL == DROPOUT == DENSE

  #1st Block
  model.add(layers.Conv2D(32,(3,3),padding='same',input_shape=x_train.shape[1:],activation='relu'))
  model.add(layers.BatchNormalization())  #To prevent vanishing gradient problem (Which arrises due to inverse covariate shift)
  model.add(layers.Conv2D(32,(3,3),padding='same',activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPool2D(pool_size=(2,2)))
  model.add(layers.Dropout(0.2))

  #2nd Block
  model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Dropout(0.3))

  #3rd Block
  model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Dropout(0.4))


  #Dense
  model.add(layers.Flatten())
  model.add(layers.Dense(nb_classes,activation = 'softmax'))
  

  return model

  model.summary()



In [0]:
# Initialize the train and test data and compile the model

x_train, y_train ,x_test, y_test = load_data()
model = build_model()

model.compile(loss='categorical_crossentropy',optimizer = 'RMSprop',metrics=['accuracy'])

In [133]:
# Train the model with Augmented Data


#.fit_generator means that there is an underlying function generating the input data instead of having a fixed input data to be fed in the model (which is ImageDataGenerator in this case which augments your data. This data augmentation process keeps on happening throughout the training;data will be augmented for the start of every training epoch)
model.fit_generator(datagen.flow(x_train,y_train,batch_size=64) ,epochs=50,verbose=1,validation_data=(x_test,y_test))

Epoch 1/50
782/782 [==============================] - 43s 55ms/step - loss: 1.9973 - accuracy: 0.3784 - val_loss: 2.1706 - val_accuracy: 0.4206
Epoch 2/50
782/782 [==============================] - 43s 55ms/step - loss: 1.5538 - accuracy: 0.5016 - val_loss: 1.3657 - val_accuracy: 0.5543
Epoch 3/50
782/782 [==============================] - 43s 55ms/step - loss: 1.4194 - accuracy: 0.5536 - val_loss: 1.2589 - val_accuracy: 0.5916
Epoch 4/50
782/782 [==============================] - 43s 55ms/step - loss: 1.2742 - accuracy: 0.5892 - val_loss: 1.2889 - val_accuracy: 0.5921
Epoch 5/50
782/782 [==============================] - 43s 54ms/step - loss: 1.1509 - accuracy: 0.6220 - val_loss: 0.9817 - val_accuracy: 0.6772
Epoch 6/50
782/782 [==============================] - 43s 55ms/step - loss: 1.0710 - accuracy: 0.6458 - val_loss: 0.8995 - val_accuracy: 0.7164
Epoch 7/50
782/782 [==============================] - 42s 54ms/step - loss: 1.0064 - accuracy: 0.6592 - val_loss: 0.9287 - val_accuracy:

In [0]:
#save to disk. Save Model in .json and weights in .h5 file
model_json = model.to_json()

with open('cifar10_architec.json','w') as json_file:  
  json_file.write(model_json)

model.save_weights('cifar10_weights.h5')

In [136]:
# Evaluate the model
scores = model.evaluate(x_test,y_test,batch_size=128,verbose=1)
print(f'Test results:{scores[0]}  and Loss:{scores[0]}')

79/79 [==============================] - 1s 11ms/step - loss: 0.5597 - accuracy: 0.8207
Test results:0.5596895217895508  and Loss:0.5596895217895508


In [1]:
#Predict images using saved weights

#load model
model_archi = 'cifar10_architec.json'
model_weights = 'cifar10_weights.h5'
model = model_from_json(open(model_archi).read())
model.load_weights(model_weights)

NameError: ignored

In [0]:
# Compile the model

optim = SGD()
model.compile(loss='categorical_crossentropy',optimizer=optim,metrics=['accuracy'])

In [139]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import matplotlib.pyplot as plt

# load the image. Place the image directory as input to img
img = load_img('/content/frog.jpg')     

# convert to numpy array
img_array = img_to_array(img)

# Resize input image to feed into network
imgs = [resize(img_array, (32, 32)).astype("float32")]
imgs = np.array(imgs) / 255
print("imgs.shape:", imgs.shape)

imgs.shape: (1, 32, 32, 3)


In [135]:
predictions = model.predict_classes(imgs)
print("predictions:", label_names[str(predictions[0])])

predictions: Frog
